<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/KoBERT_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.7 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 
     |████████████████████████████████| 6.6 MB 54.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel

# kobert pretrained model을 불러옴.

model = BertModel.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

In [15]:
# 모델의 파라미터 개수 출력

sum(p.numel() for p in model.parameters() if p.requires_grad)

92186880

In [16]:
# Bert 인코더 내 여러 모듈들 중 self-attention 모듈을 선택한 모습

module = model.encoder.layer[0].attention.self
print(module)

BertSelfAttention(
  (query): Linear(in_features=768, out_features=768, bias=True)
  (key): Linear(in_features=768, out_features=768, bias=True)
  (value): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [17]:
# Self-Attention은 Query, Key, Value로 이루어져 있음.
# key에 random_unstructured prunding 적용

module = model.encoder.layer[0].attention.self.key
prune.random_unstructured(module, name="weight", amount=0.3)

Linear(in_features=768, out_features=768, bias=True)

In [18]:
# wieght_orig 텐서에 weight_mask가 덧입혀져 Pruning이 적용된 새로운 weight 텐서가 모듈의 속성으로 저장됨.

print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
        -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
         8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -1.0340e-08,
        -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
        -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
        -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-08,
         1.1002e-07, -6.4796e-08, -1.4740e-07, -1.9011e-08,  2.2245e-07,
        -1.4329e-07, -2.8459e-08,  7.1192e-08, -5.4687e-08, -1.0020e-07,
        -1.4137e-07, -4.4915e-08, -2.6025e-08, -2.9728e-07, -8.7707e-08,
        -3.9435e-08, -5.6708e-08, -4.0600e-08,  1.3973e-07, -2.7347e-07,
        -1.3187e-07,  1.3556e-08, -1.4675e-08,  4.6401e-08, -5.2453e-08,
        -6.1351e-08, -8.3668e-08,  1.2962e-07,  2.1639e-07,  7.9439e-08,
        -4.9919e-08, -6.5474e-08,  1.9281e-08, -4.2483e-08,  1.6391e-07,
        -3.8939e-07

In [19]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[0., 1., 0.,  ..., 1., 0., 1.],
        [0., 1., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]))]


In [20]:
print(module.weight)

tensor([[ 0.0000,  0.0427, -0.0000,  ..., -0.0344,  0.0000,  0.0277],
        [ 0.0000,  0.0752, -0.0000,  ...,  0.0860,  0.0000,  0.0289],
        [-0.0000, -0.0000, -0.0000,  ..., -0.0316,  0.0000, -0.0000],
        ...,
        [ 0.0000, -0.0000, -0.0924,  ..., -0.0338, -0.0250, -0.1278],
        [ 0.0134,  0.0512,  0.0694,  ..., -0.0000, -0.0000,  0.0294],
        [-0.0243, -0.0592, -0.0535,  ..., -0.0318, -0.0714,  0.0376]],
       grad_fn=<MulBackward0>)


In [21]:
# ln_structured를 사용하여 ln 노름이라는 기준을 가지고 중요하지 않은 30%를 제거함.

prune.ln_structured(module, name='weight', amount=0.3, n=2, dim=1)

print(list(module.named_buffers()))

[('weight_mask', tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.]]))]


In [22]:
# prune.remove 메소드를 통해 파라미터를 영구적으로 Pruning하지 않더라도,
# weight_mask를 활용해 매번 Pruning이 적용된다.

prune.remove(module, 'weight')

print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
        -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
         8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -1.0340e-08,
        -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
        -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
        -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-08,
         1.1002e-07, -6.4796e-08, -1.4740e-07, -1.9011e-08,  2.2245e-07,
        -1.4329e-07, -2.8459e-08,  7.1192e-08, -5.4687e-08, -1.0020e-07,
        -1.4137e-07, -4.4915e-08, -2.6025e-08, -2.9728e-07, -8.7707e-08,
        -3.9435e-08, -5.6708e-08, -4.0600e-08,  1.3973e-07, -2.7347e-07,
        -1.3187e-07,  1.3556e-08, -1.4675e-08,  4.6401e-08, -5.2453e-08,
        -6.1351e-08, -8.3668e-08,  1.2962e-07,  2.1639e-07,  7.9439e-08,
        -4.9919e-08, -6.5474e-08,  1.9281e-08, -4.2483e-08,  1.6391e-07,
        -3.8939e-07

In [23]:
model = BertModel.from_pretrained('monologg/kobert')

parameters_to_prune = ()

# Self-Attention의 key, query, value 모두 다 적용해보기
for i in range(12):
  parameters_to_prune += (
      (model.encoder.layer[i].attention.self.key, 'weight'),
      (model.encoder.layer[i].attention.self.query, 'weight'),
      (model.encoder.layer[i].attention.self.value, 'weight'),
  )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2
)